In [3]:
import random
import pandas as pd
from sage.graphs.graph_generators import graphs 


grafi = []  

def cikel(n):
    G = Graph()
    G.add_vertices(range(n))
    e = [(i, (i + 1) % n) for i in range(n)]
    G.add_edges(e)
    return G

def drevo(i):
    return [Graph(tree) for tree in graphs.trees(i)]
 
for i in range(2, 12):  
    grafi.append(cikel(i))
    grafi.extend(drevo(i))



# 2. možnost generiranja grafov - več časa in prostora potrebuje za generiranje grafov:    
# Zanka za generiranje grafov za število vozlišč od 1 do 8 (velja, da je zgornja meja (n+1), kjer je n največe število vozlišč, ki jih lahko ima graf) 
# lahko bi izbrali tudi večji n, vendar se mi v CoClacu program ustavi, saj je mu zmanjka prostora za shranjevanje

# for i in range(1, 9):
#    for G in graphs.nauty_geng(f'{i} -c'):  # 'c' pomeni povezane grafe
#        # Dodaj graf G v seznam grafov
#        grafi.append(G)


###########################################################

def SDCTD_stevilo2(g):
    h = g.complement()

    p = MixedIntegerLinearProgram(maximization = False)
    x = p.new_variable(binary = True)
    p.set_objective(sum([x[v] for v in g]))

    for v in g.vertices():
        neighbors = g.neighbors(v)  
        if neighbors:
            p.add_constraint(sum((x[w] + x[v]) for w in neighbors) >= 1)
        else:
            p.add_constraint(x[v] == 1)
            
    zaustavi = False

    for v in g.vertices():
        neighbors_complement = h.neighbors(v)  
        if neighbors_complement:  
            p.add_constraint(sum(x[w] for w in neighbors_complement) >= 1)

        else: 
            zaustavi = True
            break 

    if not zaustavi == True:
        p.solve()
        x = p.get_values(x)
        return(sum(1 for i in x.values() if i == 1))
    else:
        return None
    
############################################################ 

rezultati = []

for m in range(1, 100):
    g1 = random.choice(grafi)
    g2 = random.choice(grafi)
    # direktni produkt grafov g1 in g2
    g3 = g1.tensor_product(g2)

    # Izračun dominacijskega števila za vsak graf
    dominacijsko_stevilo_graf1 = SDCTD_stevilo2(g1)
    dominacijsko_stevilo_graf2 = SDCTD_stevilo2(g2)
    dominacijsko_stevilo_graf3 = SDCTD_stevilo2(g3)

    # Dodajanje rezultatov v seznam
    rezultati.append({
        'Izbira grafov': f'Izbira {m}',  # Tukaj smo popravili
        'SDCTD_graf1': dominacijsko_stevilo_graf1,
        'SDCTD_graf2': dominacijsko_stevilo_graf2,
        'SDCTD_direktni': dominacijsko_stevilo_graf3
    })

df = pd.DataFrame(rezultati)

# Definicija funkcij
def neenacba(row):
    if row['SDCTD_graf1'] is None or row['SDCTD_graf2'] is None:
        return True
    # Preveri, če velja pogoj za produkt in kartezično vrednost
    if 3 * row['SDCTD_graf1'] * row['SDCTD_graf2'] >=  row['SDCTD_direktni']:
        return True
    return False

# Doda stolpca
df['neenačba'] = df.apply(neenacba, axis=1)

# Prešteje koliko False vrednosti je v tabeli
false_neenacba = (df['neenačba'] == False).sum()

print(f"Število False v stolpcu neenacba: {false_neenacba}")

print(df)



Število False v stolpcu neenacba: 0
   Izbira grafov SDCTD_graf1 SDCTD_graf2 SDCTD_direktni  neenačba
0       Izbira 1           3           3             19      True
1       Izbira 2           2           3             15      True
2       Izbira 3        None           2              4      True
3       Izbira 4           4           4             31      True
4       Izbira 5           4           4             20      True
..           ...         ...         ...            ...       ...
94     Izbira 95           3           5             15      True
95     Izbira 96           3           4             20      True
96     Izbira 97           4           3              8      True
97     Izbira 98           4           4             28      True
98     Izbira 99           3           4             12      True

[99 rows x 5 columns]
